In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential


In [2]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url,  cache_dir='.', untar=True)
# cache_dir indicates where to download data. I specified . which means current directory
# untar true will unzip it


228818944/228813984 [==============================] - 199s 1us/step


In [3]:
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

WindowsPath('datasets/flower_photos')

In [4]:
list(data_dir.glob('*/*.jpg'))[:5]


[WindowsPath('datasets/flower_photos/daisy/100080576_f52e8ee070_n.jpg'),
 WindowsPath('datasets/flower_photos/daisy/10140303196_b88d3d6cec.jpg'),
 WindowsPath('datasets/flower_photos/daisy/10172379554_b296050f82_n.jpg'),
 WindowsPath('datasets/flower_photos/daisy/10172567486_2748826a8b.jpg'),
 WindowsPath('datasets/flower_photos/daisy/10172636503_21bededa75_n.jpg')]

In [5]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

3670


In [6]:
roses = list(data_dir.glob('roses/*'))
roses[:5]

[WindowsPath('datasets/flower_photos/roses/10090824183_d02c613f10_m.jpg'),
 WindowsPath('datasets/flower_photos/roses/102501987_3cdb8e5394_n.jpg'),
 WindowsPath('datasets/flower_photos/roses/10503217854_e66a804309.jpg'),
 WindowsPath('datasets/flower_photos/roses/10894627425_ec76bbc757_n.jpg'),
 WindowsPath('datasets/flower_photos/roses/110472418_87b6a3aa98_m.jpg')]

In [7]:
flowers_images_dict = {
    'roses': list(data_dir.glob('roses/*')),
    'daisy': list(data_dir.glob('daisy/*')),
    'dandelion': list(data_dir.glob('dandelion/*')),
    'sunflowers': list(data_dir.glob('sunflowers/*')),
    'tulips': list(data_dir.glob('tulips/*')),
}

In [8]:
flowers_labels_dict = {
    'roses': 0,
    'daisy': 1,
    'dandelion': 2,
    'sunflowers': 3,
    'tulips': 4,
}

In [9]:
flowers_images_dict['roses'][:5]

[WindowsPath('datasets/flower_photos/roses/10090824183_d02c613f10_m.jpg'),
 WindowsPath('datasets/flower_photos/roses/102501987_3cdb8e5394_n.jpg'),
 WindowsPath('datasets/flower_photos/roses/10503217854_e66a804309.jpg'),
 WindowsPath('datasets/flower_photos/roses/10894627425_ec76bbc757_n.jpg'),
 WindowsPath('datasets/flower_photos/roses/110472418_87b6a3aa98_m.jpg')]

In [10]:
str(flowers_images_dict['roses'][0])

'datasets\\flower_photos\\roses\\10090824183_d02c613f10_m.jpg'

In [11]:
img = cv2.imread(str(flowers_images_dict['roses'][0]))

In [12]:
cv2.resize(img, (180, 180)).shape

(180, 180, 3)

In [13]:
X, y = [], []

for flower_name, images in flowers_images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img, (180, 180))
        X.append(resized_img)
        y.append(flowers_labels_dict[flower_name])


In [14]:
X = np.array(X)
y = np.array(y)

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [16]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255


In [17]:
num_classes = 5

model = Sequential([
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits=True),
              metrics=['accuracy'])

model.fit(X_train_scaled, y_train, epochs=30)


Epoch 1/30
86/86 [==============================] - 26s 277ms/step - loss: 1.3546 - accuracy: 0.4186
Epoch 2/30
86/86 [==============================] - 24s 275ms/step - loss: 1.0228 - accuracy: 0.5945
Epoch 3/30
86/86 [==============================] - 23s 271ms/step - loss: 0.8551 - accuracy: 0.6621
Epoch 4/30
86/86 [==============================] - 23s 272ms/step - loss: 0.6686 - accuracy: 0.7551
Epoch 5/30
86/86 [==============================] - 23s 272ms/step - loss: 0.4924 - accuracy: 0.8219
Epoch 6/30
86/86 [==============================] - 24s 275ms/step - loss: 0.2904 - accuracy: 0.9041
Epoch 7/30
86/86 [==============================] - 24s 275ms/step - loss: 0.1553 - accuracy: 0.9531
Epoch 8/30
86/86 [==============================] - 24s 276ms/step - loss: 0.0899 - accuracy: 0.9742
Epoch 9/30
86/86 [==============================] - 24s 276ms/step - loss: 0.0557 - accuracy: 0.9847
Epoch 10/30
86/86 [==============================] - 24s 279ms/step - loss: 0.0440 - accura

In [18]:
model.evaluate(X_test_scaled, y_test)


29/29 [==============================] - 2s 71ms/step - loss: 2.4147 - accuracy: 0.6307


[2.4146652221679688, 0.6307189464569092]

Improve Test Accuracy Using Data Augmentation


In [20]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomRotation(0.1),
        layers.experimental.preprocessing.RandomZoom(0.1),
    ]
)


Train the model using data augmentation and a drop out layer


In [21]:
num_classes = 5

model = Sequential([
    data_augmentation,
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits=True),
              metrics=['accuracy'])

model.fit(X_train_scaled, y_train, epochs=30)


Epoch 1/30
86/86 [==============================] - 28s 315ms/step - loss: 1.3494 - accuracy: 0.3939
Epoch 2/30
86/86 [==============================] - 27s 316ms/step - loss: 1.0257 - accuracy: 0.5876
Epoch 3/30
86/86 [==============================] - 27s 316ms/step - loss: 0.9361 - accuracy: 0.6272
Epoch 4/30
86/86 [==============================] - 27s 315ms/step - loss: 0.8496 - accuracy: 0.6715
Epoch 5/30
86/86 [==============================] - 27s 316ms/step - loss: 0.7798 - accuracy: 0.7042
Epoch 6/30
86/86 [==============================] - 27s 315ms/step - loss: 0.7436 - accuracy: 0.7238
Epoch 7/30
86/86 [==============================] - 27s 316ms/step - loss: 0.6837 - accuracy: 0.7373
Epoch 8/30
86/86 [==============================] - 27s 317ms/step - loss: 0.6433 - accuracy: 0.7536
Epoch 9/30
86/86 [==============================] - 27s 316ms/step - loss: 0.6045 - accuracy: 0.7762
Epoch 10/30
86/86 [==============================] - 27s 316ms/step - loss: 0.5655 - accura

In [22]:
model.evaluate(X_test_scaled, y_test)


29/29 [==============================] - 2s 70ms/step - loss: 1.1854 - accuracy: 0.7407


[1.185433268547058, 0.7407407164573669]